In [1]:
# Init and import

In [2]:
import numpy as np
import pandas as pd

In [3]:
data_source = 'artwork_data.csv'
df = pd.read_csv(
    data_source, 
    index_col='id',
    usecols=['id', 'artist', 'title', 'medium', 'year', 'acquisitionYear', 'width', 'height', 'units']
)
print(df.shape)
print(df.head())

(69201, 8)
              artist                                              title  \
id                                                                        
1035   Blake, Robert  A Figure Bowing before a Seated Old Man with h...   
1036   Blake, Robert  Two Drawings of Frightened Figures, Probably f...   
1037   Blake, Robert  The Preaching of Warning. Verso: An Old Man En...   
1038   Blake, Robert     Six Drawings of Figures with Outstretched Arms   
1039  Blake, William  The Circle of the Lustful: Francesca da Rimini...   

                                                 medium  year  \
id                                                              
1035  Watercolour, ink, chalk and graphite on paper....   NaN   
1036                                  Graphite on paper   NaN   
1037        Graphite on paper. Verso: graphite on paper  1785   
1038                                  Graphite on paper   NaN   
1039                            Line engraving on paper  1826   

      a

/home/robota/propro/pluralsight_code/11_pandas_fundamentals/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Aggregate

In [5]:
for artist, sub_df in df.groupby('artist'):
    if sub_df.shape[0] > 100:
        first_acquisition = sub_df['acquisitionYear'].min()
        print(artist, first_acquisition, sub_df.shape[0])

Abrahams, Ivor 1975.0 108
Arnatt, Keith 1973.0 133
Barlow, Francis 1971.0 109
Beuys, Joseph 1972.0 578
Blake, William 1878.0 180
British (?) School 1997.0 388
Burne-Jones, Sir Edward Coley, Bt 1900.0 138
Caulfield, Patrick 1967.0 124
Constable, John 1847.0 249
Cozens, Alexander 1923.0 237
Dance-Holland, Sir Nathaniel 1955.0 169
Daniell, William 1979.0 612
Davis, John Scarlett 1935.0 104
Ferrari, León 2009.0 199
Flanagan, Barry 1969.0 162
Flaxman, John 1922.0 287
Frink, Dame Elisabeth 1953.0 125
Gabo, Naum 1958.0 132
Gainsborough, Thomas 1847.0 158
Gill, Eric 1920.0 128
Hamilton Finlay, Ian 1974.0 164
Hamilton, Richard 1964.0 138
Hepworth, Dame Barbara 1950.0 123
Hockney, David 1963.0 111
House, Gordon 1975.0 110
Jones, George 1847.0 1046
Kippenberger, Martin 2005.0 109
Kitaj, R.B. 1963.0 179
Kossoff, Leon 1963.0 116
Moore, Henry, OM, CH 1939.0 623
Nolan, Sir Sidney 1951.0 135
Paolozzi, Sir Eduardo 1958.0 385
Pasmore, Victor 1941.0 135
Phillips, Esq Tom 1971.0 274
Piper, John 1940.0 183

In [6]:
# Transform

In [7]:
def fill_missing_df(df, col, min_percentage=0.5, min_num=10):
    v_list = df[col].value_counts()
    if v_list[0] >= min_num and v_list[0] * 1.0 / df.shape[0] >= min_percentage:
        filled_col = df[col].fillna(v_list.index[0])
        return df.assign(**{col: filled_col})
    return df

In [8]:
artist_list = pd.unique(df['artist'])
filled_df = pd.concat(fill_missing_df(sub_df, 'medium') for _, sub_df in df.groupby('artist'))
print(filled_df['medium'].head())

id
6652                                      Oil paint on oak
102939    Burlap, cotton gauze, hemp rope, nylon and sisal
102938                                     Sisal and metal
103642                                     Sisal and steel
103644                                    Burlap and resin
Name: medium, dtype: object


In [9]:
# Helper Transform method

In [10]:
def fill_missing_values(values, min_percentage=0.5, min_num=10):
    v_list = values.value_counts()
    if v_list.shape[0] >= min_num and v_list.shape[0] * 1.0 / values.shape[0] >= min_percentage:
        return values.fillna(v_list.index[0])
    return values

In [11]:
filled_df_bis = df.assign(medium=df.groupby('artist')['medium'].transform(fill_missing_values))
print(filled_df_bis['medium'].head())

id
1035    Watercolour, ink, chalk and graphite on paper....
1036                                    Graphite on paper
1037          Graphite on paper. Verso: graphite on paper
1038                                    Graphite on paper
1039                              Line engraving on paper
Name: medium, dtype: object


In [12]:
# Helper Aggregate method

In [13]:
first_acquisition_years = df.groupby('artist')['acquisitionYear'].agg(np.min)
first_acquisition_years = df.groupby('artist')['acquisitionYear'].min()  ## alias shortcut
print(first_acquisition_years.head())

artist
?British School           1927.0
Abakanowicz, Magdalena    2009.0
Abbey, Edwin Austin       1924.0
Abbott, Berenice          2010.0
Abbott, Lemuel Francis    1885.0
Name: acquisitionYear, dtype: float64


In [14]:
# Helper Filter method

In [15]:
duplicated_titles_df = df.groupby('title').filter(lambda x: len(x.index) > 1)
print(duplicated_titles_df['title'].value_counts().head())

[title not known]    3572
Blank                3031
[blank]              2482
[no title]           1874
Untitled              639
Name: title, dtype: int64
